In [3]:
from allennlp.predictors.predictor import Predictor
import numpy as np
import glob

import sys
sys.path.append('../src')

from misc import open_dict, save_dict, get_file_names

In [13]:
# ##### ProppLearner from gold standard coreference chains & labels #####
# tokensDir= "../data/ProppLearner/tokenized/"
# parsesDir = '../intermediate/ProppLearner/parses/'
# fileNames = get_file_names('data/ProppLearner/corefs_gold_new_format/', '.p')

##### Propp Learner from allenNLP coreference chains & approximate labels #####
tokensDir= "../data/ProppLearner/tokenized/"
parsesDir = '../intermediate/ProppLearner/parses/'
fileNames = get_file_names('../data/ProppLearner/corefs_allen/', '.p')

# ##### LitBank from gold standard coreference chains #####
# tokensDir = "../data/LitBank/tokenized_shortened/"
# parsesDir = '../intermediate/LitBank/parses_shortened/'
# fileNames = get_file_names('data/LitBank/corefs_gold_new_format/', '.p')

# ##### LitBank from AllenNLP coreference chains #####
# tokensDir = "../data/LitBank/tokenized/"
# parsesDir = '../intermediate/LitBank/parses/'
# fileNames = get_file_names('data/LitBank/corefs_allen/', '.p')


depParseLoc = parsesDir + 'deps-biaffine/'
nerParseLoc = parsesDir + 'NER-elmo/'
openParseLoc = parsesDir + 'openIE/'
srlParseLoc = parsesDir + 'SRL-bert/'

constParseLoc = parsesDir + 'constituency/'

In [ ]:
predictorSRL = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")
predictorDep = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")
predictorNER = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-elmo.2021-02-12.tar.gz")
predictorOpen = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

In [ ]:
for num, fileName in enumerate(fileNames):

    document = open_dict(tokensDir + fileName + '.p')

    # do semantic relation parse of each sentence in document
    srlParses = []
    for sent in document['sents']:
        sTokens = sent['tokens']
        srlParse = predictorSRL.predict_tokenized(tokenized_sentence = sTokens)
        srlParses.append(srlParse)

    srlParseDict = {'parses': srlParses, 'document':document}
    save_dict(srlParseDict, srlParseLoc + fileName + '.p')

    # do semantic relation parse of each sentence in document
    depParses = []
    for sent in document['sents']:
        s = sent['text']
        depParse = predictorDep.predict(sentence = s)
        depParses.append(depParse)

    depParseDict = {'parses': depParses, 'document':document}
    save_dict(depParseDict, depParseLoc + fileName + '.p')

    # do openIE parse of each sentence in document
    openParses = []
    for sent in document['sents']:
        s = sent['text']
        openParse = predictorOpen.predict(sentence = s)
        openParses.append(openParse)

    openParseDict = {'parses': openParses, 'document':document}
    save_dict(openParseDict, openParseLoc + fileName + '.p')

    # do semantic relation parse of each sentence in document
    nerParses = []
    for sent in document['sents']:
        s = sent['text']
        nerParse = predictorNER.predict(sentence = s)
        nerParses.append(nerParse)

    nerParseDict = {'parses': nerParses, 'document':document}
    save_dict(nerParseDict, nerParseLoc + fileName + '.p')

    print(num, fileName, 'done')


In [1]:
#### Constituency Parsing

In [7]:
predictorCP = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
for num, fileName in enumerate(fileNames):

    document = open_dict(tokensDir + fileName + '.p')

    constParses = []
    for sent in document['sents']:
        s = sent['text']
        constParse = predictorCP.predict(sentence = s)
        constParses.append(constParse)

    constParseDict = {'parses': constParses, 'document':document}
    save_dict(constParseDict, constParseLoc + fileName + '.p')


KeyboardInterrupt: 

In [15]:
constParse = open_dict('../intermediate/ProppLearner/parses/constituency/story1.p')

In [35]:
document = open_dict('../data/ProppLearner/tokenized/story1.p')

In [56]:
def get_arg0s(constParses, sentences):
    positions = []

    tokenOffset = 0

    for i, parse in enumerate(constParses['parses']):
        

        for verb in parse['verbs']:
            for j, tag in enumerate(verb['tags']):
                if tag == 'B-ARG0':
                    count = 0

                    currIndex = j

                    while currIndex < len(verb['tags']) -1:
                        currIndex += 1
                        if verb['tags'][currIndex] != 'I-ARG0':
                            break
                        count+= 1

                    positions.append([j+tokenOffset, j+count+tokenOffset])

        tokenOffset += len(sentences[i]['tokens'])

    return positions

In [57]:
def get_arg1s(constParses, sentences):
    positions = []

    tokenOffset = 0

    for i, parse in enumerate(constParses['parses']):
        

        for verb in parse['verbs']:
            for j, tag in enumerate(verb['tags']):
                if tag == 'B-ARG1':
                    count = 0

                    currIndex = j

                    while currIndex < len(verb['tags']) -1:
                        currIndex += 1
                        if verb['tags'][currIndex] != 'I-ARG1':
                            break
                        count+= 1

                    positions.append([j+tokenOffset, j+count+tokenOffset])

        tokenOffset += len(sentences[i]['tokens'])

    return positions

In [58]:
def get_arg2s(constParses, sentences):
    positions = []

    tokenOffset = 0

    for i, parse in enumerate(constParses['parses']):
        

        for verb in parse['verbs']:
            for j, tag in enumerate(verb['tags']):
                if tag == 'B-ARG2':
                    count = 0

                    currIndex = j

                    while currIndex < len(verb['tags']) -1:
                        currIndex += 1
                        if verb['tags'][currIndex] != 'I-ARG2':
                            break
                        count+= 1

                    positions.append([j+tokenOffset, j+count+tokenOffset])

        tokenOffset += len(sentences[i]['tokens'])

    return positions

In [64]:
arg1s = get_arg1s(constParse, document['sents'])

In [65]:
for pos in arg1s:
    for i in range(pos[0], pos[-1] + 1):
        print(document['tokens'][i] + ' ', end='')

    print()

A dragon 
heavy tribute 
a lovely maiden from every house 
to go to the dragon 
her 
her 
her 
she 
her 
his house 
the princess 
a little dog that had followed her to the dragon 's lair 
her letter 
it 
the answer 
to find out who in this world was stronger than the dragon 
who in this world was stronger than the dragon 
who in this world 
The princess 
to question him 
that a tanner in the city of Kiev was stronger than he 
a tanner in the city of Kiev 
this 
Nikita the Tanner 
him 
her 
this letter 
Nikita the Tanner 
his land 
the princess 
hides 
twelve hides 
that the tsar in person had come to see him 
the tsar in person 
him 
to tremble with fear 
he 
his hands 
the twelve hides 
him 
to go forth against the dragon 
five thousand little children 
them 
that their tears would move him to pity 
him 
The little children 
him 
fight the dragon 
the dragon 
to shed tears when he saw theirs 
tears 
theirs 
twelve thousand pounds of hemp 
it 
around himself 
him 
battle 
Nikita 
himse

In [66]:
arg0s

[[6, 6],
 [22, 22],
 [33, 36],
 [43, 43],
 [43, 43],
 [43, 43],
 [66, 66],
 [73, 73],
 [77, 77],
 [77, 77],
 [84, 85],
 [89, 90],
 [92, 94],
 [105, 106],
 [115, 115],
 [127, 128],
 [127, 128],
 [143, 146],
 [143, 146],
 [153, 153],
 [167, 168],
 [167, 168],
 [184, 184],
 [192, 192],
 [209, 210],
 [214, 214],
 [214, 214],
 [214, 214],
 [228, 228],
 [240, 241],
 [240, 241],
 [240, 241],
 [247, 249],
 [247, 249],
 [266, 266],
 [266, 266],
 [279, 279],
 [282, 285],
 [304, 304],
 [315, 318],
 [322, 322],
 [322, 322],
 [332, 332],
 [332, 332],
 [341, 341],
 [332, 332],
 [348, 349],
 [356, 358],
 [364, 364],
 [364, 364],
 [373, 374],
 [373, 374],
 [380, 380],
 [384, 384],
 [384, 384],
 [384, 384],
 [404, 405],
 [384, 384],
 [384, 384],
 [427, 428],
 [444, 444],
 [448, 448],
 [459, 459],
 [459, 459],
 [467, 468],
 [472, 472],
 [467, 468],
 [467, 468],
 [487, 487],
 [503, 503],
 [508, 509],
 [508, 509],
 [540, 540],
 [554, 554],
 [573, 573],
 [577, 577],
 [584, 584],
 [584, 584],
 [601, 602],
 